--- Day 20: Pulse Propagation ---

With your help, the Elves manage to find the right parts and fix all of the machines. Now, they just need to send the command to boot up the machines and get the sand flowing again.

The machines are far apart and wired together with long cables. The cables don't connect to the machines directly, but rather to communication modules attached to the machines that perform various initialization tasks and also act as communication relays.

Modules communicate using pulses. Each pulse is either a high pulse or a low pulse. When a module sends a pulse, it sends that type of pulse to each module in its list of destination modules.

There are several different types of modules:

Flip-flop modules (prefix %) are either on or off; they are initially off. If a flip-flop module receives a high pulse, it is ignored and nothing happens. However, if a flip-flop module receives a low pulse, it flips between on and off. If it was off, it turns on and sends a high pulse. If it was on, it turns off and sends a low pulse.

Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected input modules; they initially default to remembering a low pulse for each input. When a pulse is received, the conjunction module first updates its memory for that input. Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.

There is a single broadcast module (named broadcaster). When it receives a pulse, it sends the same pulse to all of its destination modules.

Here at Desert Machine Headquarters, there is a module with a single button on it called, aptly, the button module. When you push the button, a single low pulse is sent directly to the broadcaster module.

After pushing the button, you must wait until all pulses have been delivered and fully handled before pushing it again. Never push the button if modules are still processing pulses.

Pulses are always processed in the order they are sent. So, if a pulse is sent to modules a, b, and c, and then module a processes its pulse and sends more pulses, the pulses sent to modules b and c would have to be handled first.

The module configuration (your puzzle input) lists each module. The name of the module is preceded by a symbol identifying its type, if any. The name is then followed by an arrow and a list of its destination modules. For example:

```
broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
```

In this module configuration, the broadcaster has three destination modules named a, b, and c. Each of these modules is a flip-flop module (as indicated by the % prefix). a outputs to b which outputs to c which outputs to another module named inv. inv is a conjunction module (as indicated by the & prefix) which, because it has only one input, acts like an inverter (it sends the opposite of the pulse type it receives); it outputs to a.

By pushing the button once, the following pulses are sent:

```
button -low-> broadcaster
broadcaster -low-> a
broadcaster -low-> b
broadcaster -low-> c
a -high-> b
b -high-> c
c -high-> inv
inv -low-> a
a -low-> b
b -low-> c
c -low-> inv
inv -high-> a
```

After this sequence, the flip-flop modules all end up off, so pushing the button again repeats the same sequence.

Here's a more interesting example:

```
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
```

This module configuration includes the broadcaster, two flip-flops (named a and b), a single-input conjunction module (inv), a multi-input conjunction module (con), and an untyped module named output (for testing purposes). The multi-input conjunction module con watches the two flip-flop modules and, if they're both on, sends a low pulse to the output module.

Here's what happens if you push the button once:

```
button -low-> broadcaster
broadcaster -low-> a
a -high-> inv
a -high-> con
inv -low-> b
con -high-> output
b -high-> con
con -low-> output
```

Both flip-flops turn on and a low pulse is sent to output! However, now that both flip-flops are on and con remembers a high pulse from each of its two inputs, pushing the button a second time does something different:

```
button -low-> broadcaster
broadcaster -low-> a
a -low-> inv
a -low-> con
inv -high-> b
con -high-> output
```

Flip-flop a turns off! Now, con remembers a low pulse from module a, and so it sends only a high pulse to output.

Push the button a third time:

```
button -low-> broadcaster
broadcaster -low-> a
a -high-> inv
a -high-> con
inv -low-> b
con -low-> output
b -low-> con
con -high-> output
```

This time, flip-flop a turns on, then flip-flop b turns off. However, before b can turn off, the pulse sent to con is handled first, so it briefly remembers all high pulses for its inputs and sends a low pulse to output. After that, flip-flop b turns off, which causes con to update its state and send a high pulse to output.

Finally, with a on and b off, push the button a fourth time:

```
button -low-> broadcaster
broadcaster -low-> a
a -low-> inv
a -low-> con
inv -high-> b
con -high-> output
```

This completes the cycle: a turns off, causing con to remember only low pulses and restoring all modules to their original states.

To get the cables warmed up, the Elves have pushed the button 1000 times. How many pulses got sent as a result (including the pulses sent by the button itself)?

In the first example, the same thing happens every time the button is pushed: 8 low pulses and 4 high pulses are sent. So, after pushing the button 1000 times, 8000 low pulses and 4000 high pulses are sent. Multiplying these together gives 32000000.

In the second example, after pushing the button 1000 times, 4250 low pulses and 2750 high pulses are sent. Multiplying these together gives 11687500.

Consult your module configuration; determine the number of low pulses and high pulses that would be sent after pushing the button 1000 times, waiting for all pulses to be fully handled after each push of the button. What do you get if you multiply the total number of low pulses sent by the total number of high pulses sent?

Your puzzle answer was 1020211150.

In [ ]:
from collections import defaultdict, deque
import re

def read_input(filePath):
    with open(filePath, 'r') as file:
        return file.read()
    
def read_modules(text):
    module_pattern = r'([%&]{0,1})(\w+)\s->\s(.*)'
    dests_pattern = r'(\w+),*'
    lines = re.findall(module_pattern, text)
    return {key: (type, re.findall(dests_pattern, dests)) for (type, key, dests) in lines}

def initialize_states(modules):
    #
    #  Flip-flop module (type = %):
    #
    #   state: 0/1 (initially 0)
    #
    #  Conjunction module (type = &):
    #
    #   input1: 0/1 (default is 0)
    #   input2: 0/1 (default is 0)
    #   ...
    #
    sources = defaultdict(list)
    for (module_key, (_, dest_keys)) in modules.items():
        for dest_key in dest_keys:
            sources[dest_key].append(module_key)
    module_states = {}
    for (module_key, (module_type, dest_keys)) in modules.items():
        if module_type == '':
            module_states[module_key] = None
        elif module_type == '%':
            module_states[module_key] = {'state': 0}
        elif module_type == '&':
            module_states[module_key] = {source: 0 for source in sources[module_key]}
    return module_states

def process_pulse(module_type, module_state, source_key, incoming_pulse):
    #
    #   Flip-flop module (type = %):
    #
    #   - Low pulse => the state is flipped and new state is returned
    #   - High pulse => the pulse is "swallowed"
    #
    #  Conjunction module (type = &):
    #
    #   - Remember the last pulse received from each input.
    #   - If all inputs are high, send a low pulse.
    #   - Otherwise, send a high pulse.
    #
    if (module_type == None):
        return incoming_pulse
    elif (module_type == '%'):
        if incoming_pulse == 0:
            module_state['state'] = 1 - module_state['state']
            return module_state['state']
        else:
            return None
    elif (module_type == '&'):
        module_state[source_key] = incoming_pulse
        if all(module_state.values()):
            return 0
        else:
            return 1
    return incoming_pulse

def process_pulses(modules, module_states):
    queue = deque([('button', 0, 'broadcaster')])
    while queue:
        source_key, incoming_pulse, module_key = queue.popleft()
        if module_key in modules:
            module_type = modules[module_key][0]
            module_state = module_states[module_key]
            outgoing_pulse = process_pulse(module_type, module_state, source_key, incoming_pulse)
            if outgoing_pulse != None:
                for dest_key in modules[module_key][1]:
                    queue.append((module_key, outgoing_pulse, dest_key))
        yield (source_key, incoming_pulse, module_key)

input = read_input('sample2.txt')
modules = read_modules(input)
module_states = initialize_states(modules)

previous_press = 0
number_of_low_pulses = 0
number_of_high_pulses = 0
for button_press in range(1000):
    print('----------------')
    print(button_press, module_states)
    for src, pulse, dest in process_pulses(modules, module_states):
        print(button_press, f'{src} -{'low' if pulse == 0 else 'high'}-> {dest}')
        if pulse == 0:
            number_of_low_pulses += 1
        else:
            number_of_high_pulses += 1

print('----------------')
print(number_of_low_pulses)
print(number_of_high_pulses)
print(number_of_low_pulses * number_of_high_pulses)


--- Part Two ---

The final machine responsible for moving the sand down to Island Island has a module attached named rx. The machine turns on when a single low pulse is sent to rx.

Reset all modules to their default states. Waiting for all pulses to be fully handled after each button press, what is the fewest number of button presses required to deliver a single low pulse to the module named rx?

Your puzzle answer was 238815727638557.

In [ ]:
from math import lcm

def get_cycle_length(modules, module_states, module_key):
    last_press = 0
    for button_press in range(1, 10000):
        for _, _, _ in process_pulses(modules, module_states):
            if module_states['nc'][module_key] and button_press != last_press:
                last_press = button_press
                yield button_press

input = read_input('input.txt')
modules = read_modules(input)
module_states = initialize_states(modules)

#
#   This task is not possible to simulate using the previous method. Instead, if we look at our 
#   specific input we see that rx only has one step &nc -> rx, where nc is a conjunction module with
#   some number of inputs.
#
#       &lk -> nc
#       &fn -> nc
#       &fh -> nc
#       &hh -> nc
#
#       &nc -> rx
#
#   The inputs to nc must all be 'high' at the same time for the output of nc to be 'low'.
#   We will look at the cycle lengths for the inputs of nc, meaning the number of button presses
#   required for each input to be 'high' again. The final answer will be the LCM of these cycle lengths.
#
cycle_lengths = []
for module_key in ['lk', 'fn', 'fh', 'hh']:
    button_presses = [v for i, v in enumerate(get_cycle_length(modules, module_states, module_key)) if i < 2]
    cycle_lengths.append(button_presses[1] - button_presses[0])

print(lcm(*cycle_lengths))